# Reading Data in Database Files

In [ ]:
# Imports
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

In [ ]:
# Download the database
!wget -P ./ https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/classic_rock.db

In [ ]:
# Initialize path to SQLite databasejdbc:sqlite:/C:/__tmp/test/sqlite/jdbcTest.db
path = 'data/classic_rock.db'
con = sq3.Connection(path)

# We now have a live connection to our SQL database

In [ ]:
# Now that we've got a connection to our database, we can perform queries, and load their results in as Pandas DataFrames

# Write the query
query = '''
SELECT * 
FROM rock_songs;
'''

# Execute the query
observations = pds.read_sql(query, con)

observations.head()

In [ ]:
# We can also run any supported SQL query
# Write the query
query = '''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

# Execute the query
observations = pds.read_sql(query, con)

observations.head()

Some common paramters that can be used to read in SQL data with formatting:

 - coerce_float: Attempt to force numbers into floats
 - parse_dates: List of columns to parse as dates
 - chunksize: Number of rows to include in each chunk

In [ ]:
query='''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

# Execute the query
observations_generator = pds.read_sql(query,
                            con,
                            coerce_float=True, # Doesn't efefct this dataset, because floats were correctly parsed
                            parse_dates=['Release_Year'], # Parse `Release_Year` as a date
                            chunksize=5 # Allows for streaming results as a series of shorter tables
                           )

for index, observations in enumerate(observations_generator):
    if index < 5:
        print(f'Observations index: {index}'.format(index))
        display(observations)

In [ ]:
# Download the database
!wget -P ./ https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/baseball.db

In [ ]:
path = 'baseball.db'

# Create a connection, `con`, that is connected to database at `path`
con = sq3.Connection(path)

# Create a variable, `query`, containing a SQL query which reads in all data from the `` table

query = """
SELECT *
    FROM allstarfull
    ;
"""

allstar_observations = pd.read_sql(query, con)

# print(allstar_observations)

# Create a variable, tables, which reads in all data from the table sqlite_master
all_tables = pd.read_sql('SELECT * FROM sqlite_master', con)
print(all_tables)

# Pretend that you were interesting in creating a new baseball hall of fame. Join and analyze the tables to evaluate the top 3 all time best baseball players
best_query = """
SELECT playerID, sum(GP) AS num_games_played, AVG(startingPos) AS avg_starting_position
    FROM allstarfull
    GROUP BY playerID
    ORDER BY num_games_played DESC, avg_starting_position ASC
    LIMIT 3
"""
best = pd.read_sql(best_query, con)
print(best.head())

# Data Cleaning


In [ ]:
import pandas as pd
import numpy as np 

import seaborn as sns 
import matplotlib.pylab as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import norm
from scipy import stats

In [ ]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/Ames_Housing_Data1.tsv'
housing = pd.read_csv(path, sep='\t')

housing.head(10)

In [ ]:
# We can find more information about the features and types using the `info()`  method.

housing.info()

According to the output above, we have 2930 entries, 0 to 2929, as well as 81 features. The "Non-Null Count" column shows the number of non-null entries.  If the count is 2930 then there is no missing values for that particular feature. 'SalePrice' is our target or response variable and the rest of the features are our predictor variables.

We also have a mix of numerical (28 int64 and 11 float64) and object data types. 


In [ ]:
housing["SalePrice"].describe()

From the above analysis, it is important to note that the minimum value is greater than 0. Also, there is a big difference between the minimum value and the 25th percentile. It is bigger than the 75th percentile and the maximum value. This means that our data might not be normally distributed (an important assumption for linear regression analysis), so will check for normality in the Log Transform section. 

The `describe()` function reveals the statistical information about the numeric attributes. To reveal some information about our categorical (object) attributes, we can use `value_counts()` function. In this exercise, describe all categories of the 'Sale Condition' attribute.



In [ ]:
housing["Sale Condition"].value_counts()

## **Looking for Correlations**

Before starting the process of data cleaning, it is important to determine the correlation between the response variable (in our case, the sale price) and other predictor variables. Some variables may not have a significant impact on the price of the house and will not be included in the analysis. There are several methods to identify the correlation between the target variable and other features, such as pair plots, scatter plots, heat maps, and correlation matrices. In this case, we will use the `corr()` function to list the top features based on the Pearson correlation coefficient, which measures how closely two sequences of numbers are correlated. However, it's important to note that correlation coefficient can only be calculated on numerical attributes like floats and integers. Therefore, only numeric attributes will be selected for the analysis.

In [ ]:
hous_num = housing.select_dtypes(include = ['float64', 'int64'])
hous_num_corr = hous_num.corr()['SalePrice'][:-1] # -1 means that the latest row is SalePrice

# print(hous_num_corr )
top_features = hous_num_corr[abs(hous_num_corr) > 0.5].sort_values(ascending=False) #displays pearsons correlation coefficient greater than 0.5
print(f"There is {len(top_features)} strongly correlated values with SalePrice:\n{top_features}")

In [ ]:
for i in range(0, hous_num.columns.size, 5):
    sns.pairplot(data=hous_num,
                x_vars=hous_num.columns[i:i+5],
                y_vars=['SalePrice'])

## **Log Transformation**
In this section, we are going to inspect whether our 'SalePrice' data are normally distributed. The assumption of the normal distribution must be met in order to perform any type of regression analysis. There are several ways to check for this assumption, however here, we will use the visual method, by plotting the 'SalePrice' distribution using the `distplot()` function from the `seaborn` library.


In [ ]:
sp_untransformed = sns.distplot(housing['SalePrice'])

In [ ]:
# The range of skewness for a fairly symmetrical bell curve distribution is between -0.5 and 0.5; moderate skewness is -0.5 to -1.0 and 0.5 to 1.0; and highly skewed distribution is < -1.0 and > 1.0. In our case, we have ~1.7, so it is considered  highly skewed data. 

print(f"Skewness: {housing['SalePrice'].skew()}")

In [ ]:
# transform our data, so it looks more normally distributed.
log_transformed = np.log(housing['SalePrice'])

sp_transformed = sns.distplot(log_transformed)

print(f"Skewness: {log_transformed.skew()}")

Other ways to correct for skewness of the data are: Square Root Transform (`np.sqrt`) and the Box-Cox Transform (`stats.boxcox` from the `scipy stats` library).

## **Handling the Duplicates**


In [ ]:
# find duplicate rows
duplicate = housing[housing.duplicated(['PID'])]
duplicate

In [ ]:
dup_removed = housing.drop_duplicates()
dup_removed 

In [ ]:
# alternative way to check if there are any duplicated Indexes in our dataset is to use the `is_unique` method.
housing.index.is_unique

## **Handling the Missing Values**
To facilitate the identification of any missing values, pandas offers three functions: `isna()`, `isnull()`, and `notna()`.

To get an overview of the missing values in our dataset, we will use the `isnull()` function. Then, we will sum up the values using the `sum()` function, sort them with `sort_values()`, and finally plot the first 20 columns, which have the highest number of missing values, using the `bar plot` function from the `matplotlib` library.

In [ ]:
total = housing.isnull().sum().sort_values(ascending=False)
total_select = total.head(20)
total_select.plot(kind="bar", figsize = (8,6), fontsize = 10)

plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
plt.title("Total Missing Values", fontsize = 20)

In [ ]:
# All the rows, containing null values in 'Lot Frontage' feature will be dropped.
housing.dropna(subset=["Lot Frontage"])

In [ ]:
# drop the whole attribute or feature (column), that contains missing values:
housing.drop("Lot Frontage", axis=1)

In [ ]:
# replace the missing values (zero, the mean, the median, etc.)
median = housing["Lot Frontage"].median()
housing["Lot Frontage"].fillna(median, inplace = True)
housing.tail()

## **Feature Scaling**
Feature scaling is a crucial step in data transformation. Two common techniques include min-max scaling and standardization.

In [ ]:
norm_data = MinMaxScaler().fit_transform(hous_num)
norm_data

In [ ]:
scaled_data = StandardScaler().fit_transform(hous_num)
scaled_data

## **Handling the Outliers**

In statistics, an outlier refers to an observation that is significantly different from other observations. This can happen due to errors in data collection or recording, or simply because of natural variability in the data. The way we treat an outlier largely depends on the type of analysis we want to perform and the data we are working with. Outliers can have a major impact on our statistical models, and can also provide valuable insights into specific behaviours.

There are many techniques to discover outliers in our data, such as Uni-variate analysis (using one variable analysis) or Multi-variate analysis (using two or more variables). One of the easiest ways to detect an outlier is to visually inspect the data using box plots or scatter plots.

### Uni-variate Analysis
From the two plots shown, it is evident that there are some points outside the box plot area that significantly deviate from the rest of the population. The decision to keep or remove them depends on how well we understand our data and the type of analysis to be performed. In this particular case, the points outside the box plot area for 'Lot Area' and 'Sale Price' may represent actual true data points, and therefore, they should not be removed.

In [ ]:
sns.boxplot(x=housing['Lot Area'])

In [ ]:
sns.boxplot(x=housing['SalePrice'])

### Bi-variate Analysis


In [ ]:
price_area = housing.plot.scatter(x='Gr Liv Area',
                      y='SalePrice')

### Deleting the Outliers


In [ ]:
housing.sort_values(by = 'Gr Liv Area', ascending = False)[:2]

In [ ]:
outliers_dropped = housing.drop(housing.index[[1499,2181]])

new_plot = outliers_dropped.plot.scatter(x='Gr Liv Area',
                                         y='SalePrice')

## Z-score Analysis

Z-score can be used to identify outliers in a mathematical way. It is calculated by measuring the number of standard deviations by which an observation or data point is above or below the mean value of the group being observed. In other words, the Z-score is a value that measures the relationship between a specific data point and the mean and standard deviation of the entire group. Data points that are too far from the mean, typically more than 3 or less than -3 standard deviations, are considered outliers. Therefore, if the Z-score value of a data point exceeds this threshold, it will be identified as an outlier.

In [ ]:
housing['LQFSF_Stats'] = stats.zscore(housing['Low Qual Fin SF'])
housing[['Low Qual Fin SF','LQFSF_Stats']].describe().round(3)

In [ ]:
# The max value of 22.882 is further proof of the presence of outliers, as it falls well above the z-score limit of +3.

## Exploratory Data Analysis (EDA)

In [ ]:
import os
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/iris_data.csv'

data = pd.read_csv(path)
data.head()

In [ ]:
# Number of rows
print(data.shape)

# Column names
print(data.columns.tolist())

# Data types
print(data.dtypes)

In [ ]:
data['species'].value_counts()

In [ ]:
stats_df = data.describe()
stats_df

In [ ]:
stats_df.loc['range'] = stats_df.loc['max'] - stats_df.loc['min']
stats_df

In [ ]:
out_fields = ['mean','25%','50%','75%', 'range']
stats_df = stats_df.loc[out_fields]
stats_df.rename({'50%': 'median'}, inplace=True)
stats_df

In [ ]:
data.groupby('species').mean()

In [ ]:
data.groupby('species').median()

In [ ]:
# applying multiple functions at once - 2 methods

data.groupby('species').agg(['mean', 'median'])  # passing a list of recognized strings
data.groupby('species').agg([np.mean, np.median])  # passing a list of explicit aggregation functions

In [ ]:
data.columns

In [ ]:
from pprint import pprint

agg_dict = {field: ['mean', 'median'] for field in data.columns if field != 'species'}
agg_dict['petal_length'] = 'max'
pprint(agg_dict)
data.groupby('species').agg(agg_dict)

In [ ]:
ax = plt.axes()

ax.scatter(data.sepal_length, data.sepal_width)

# Label the axes
ax.set(xlabel='Sepal Length (cm)',
       ylabel='Sepal Width (cm)',
       title='Sepal Length vs Width');

In [ ]:
ax = plt.axes()
ax.hist(data.petal_length, bins=25);

ax.set(xlabel='Petal Length (cm)', 
       ylabel='Frequency',
       title='Distribution of Petal Lengths');

In [ ]:
sns.set_context('notebook')
data.plot.hist(bins=25, alpha=0.5)

In [ ]:
# To create four separate plots, use Pandas `.hist` method
sns.set_context('talk')
axList = data.hist(bins=25)

In [ ]:
# To create four separate plots, use Pandas `.hist` method
sns.set_context('notebook')

axList = data.hist(bins=25)

# Add some x- and y- labels to first column and last row
for ax in axList.flatten():
    if ax.is_last_row():
        ax.set_xlabel('Size (cm)')
        
    if ax.is_first_col():
        ax.set_ylabel('Frequency')

In [ ]:
#sns.set_context('poster')

data.boxplot(by='species');


In [ ]:
# First we have to reshape the data so there is 
# only a single measurement in each column

plot_data = (data
             .set_index('species')
             .stack()
             .to_frame()
             .reset_index()
             .rename(columns={0:'size', 'level_1':'measurement'})
            )

plot_data.head()

In [ ]:
sns.set_style('white')
sns.set_context('notebook')
sns.set_palette('dark')

f = plt.figure(figsize=(6,4))
sns.boxplot(x='measurement', y='size', 
            hue='species', data=plot_data);

In [ ]:
sns.set_context('talk')
sns.pairplot(data, hue='species');

### Feature Engineering 

In [ ]:
## Load in the Ames Housing Data
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/Ames_Housing_Data.tsv'

df = pd.read_csv(path, sep='\t')

df.info()

In [ ]:
# This is recommended by the data set author to remove a few outliers

df = df.loc[df['Gr Liv Area'] <= 4000,:]
print("Number of rows in the data:", df.shape[0])
print("Number of columns in the data:", df.shape[1])
data = df.copy() # Keep a copy our original data 
df.head()

In [ ]:
# drop columns that are not useful for ML and for which the value is unique for all observations
df.drop(['Order', 'PID'], axis=1, inplace=True)

In [ ]:
# Get a Pd.Series consisting of all the string categoricals
one_hot_encode_cols = df.dtypes[df.dtypes == object]  # filtering by string categoricals
one_hot_encode_cols = one_hot_encode_cols.index.tolist()  # list of categorical fields

df[one_hot_encode_cols].head().T

In [ ]:
# Do the one hot encoding
df = pd.get_dummies(df, columns=one_hot_encode_cols, drop_first=True)
df.describe().T

In [ ]:
df.select_dtypes(include=['object']).columns

In [ ]:
# Create a list of float colums to check for skewing
mask = data.dtypes == float
float_cols = df.select_dtypes('number').columns #data.columns[mask]

skew_limit = 0.75 # define a limit above which we will log transform
skew_vals = data[float_cols].skew()
print(skew_vals)


In [ ]:
# Showing the skewed columns
skew_cols = (skew_vals
             .sort_values(ascending=False)
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))

skew_cols

In [ ]:
# Let's look at what happens to one of these features, when we apply np.log1p visually.

# Choose a field
field = "SalePrice"

# Create two "subplots" and a "figure" using matplotlib
fig, (ax_before, ax_after) = plt.subplots(1, 2, figsize=(10, 5))

# Create a histogram on the "ax_before" subplot
df[field].hist(ax=ax_before)

# Apply a log transformation (numpy syntax) to this column
df[field].apply(np.log1p).hist(ax=ax_after)

# Formatting of titles etc. for each subplot
ax_before.set(title='before np.log1p', ylabel='frequency', xlabel='value')
ax_after.set(title='after np.log1p', ylabel='frequency', xlabel='value')
fig.suptitle('Field "{}"'.format(field));

In [ ]:
# Perform the skew transformation:

for col in skew_cols.index.values:
    if col == "SalePrice":
        continue
    df[col] = df[col].apply(np.log1p)

In [ ]:
# We now have a larger set of potentially-useful features

df.shape

In [ ]:
# There are a *lot* of variables. Let's go back to our saved original data and look at how many values are missing for each variable. 
df = data
data.isnull().sum().sort_values()

In [ ]:
smaller_df= df.loc[:,['Lot Area', 'Overall Qual', 'Overall Cond', 
                      'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
                      'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                      'Garage Cars','SalePrice']]

# Now we can look at summary statistics of the subset data
smaller_df.describe().T

In [ ]:
smaller_df.info()

In [ ]:
# There appears to be one NA in Garage Cars - we will take a simple approach and fill it with 0
smaller_df = smaller_df.fillna(0)

smaller_df.info()

In [ ]:
sns.pairplot(smaller_df, plot_kws=dict(alpha=.1, edgecolor='none'))

In [ ]:
#Separate our features from our target

X = smaller_df.loc[:,['Lot Area', 'Overall Qual', 'Overall Cond', 
                      'Year Built', 'Year Remod/Add', 'Gr Liv Area', 
                      'Full Bath', 'Bedroom AbvGr', 'Fireplaces', 
                      'Garage Cars']]

y = smaller_df['SalePrice']

X.info()

In [ ]:
data['House Style'].value_counts()

In [ ]:
pd.get_dummies(df).head()

In [ ]:
pd.get_dummies(df['House Style'], drop_first=True).head()

In [ ]:
nbh_counts = df.Neighborhood.value_counts()
nbh_counts

In [ ]:
other_nbhs = list(nbh_counts[nbh_counts <= 8].index)

other_nbhs

In [ ]:
X4 = X.copy()

X4['Neighborhood'] = df['Neighborhood'].replace(other_nbhs, 'Other')

X4.Neighborhood.value_counts()
